# 线性回归练习（Linear Regression Practice）

## 项目说明

#### 项目背景：
* 1.项目基于吴达恩教授的《机器学习》课程。
* 2.数据均来源于课程配套资料。


#### 项目目的：
* 1、使用单变量线性回归分析ex1data1并预测。
* 2、使用多变量线性回归分析ex1data2并预测。
* 3、熟悉线性回归的假设函数、代价函数。
* 4、熟练运用梯度下降和正规方程解决线性回归问题。


#### 单变量线性回归：
* 1、分析目的：通过城市人口预测快餐车的收益，帮助老板选择合适的城市进行扩张。
* 2、数据说明：ex1data1
    * 第1列：城市人口数
    * 第2列：快餐车收益，为负则表示快餐车亏本。


#### 多变量线性回归：
* 1、分析目的：通过分析俄勒冈州波特兰市的房价与房子大小、房间数量的关系，得出房价预测模型。
* 2、数据说明：ex1data2
    * 第1列：房子面积，单位：平方英尺
    * 第2列：房间数量
    * 第3列：房子售卖价格

# 导入相关数据库

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly as py
import plotly.graph_objs as go
import cufflinks as cf
from plotly.offline import iplot,init_notebook_mode
cf.go_offline(connected=True)
init_notebook_mode(connected=True)
cf.set_config_file(theme='pearl') #设置一下制图颜色

# 1.单变量线性回归

## 1）读取数据并理解

In [3]:
#读取数据并赋予列名
df = pd.read_csv('D:\Learning\python\ML\code\ex1-linear regression\ex1data1.txt',names=['population','profit'])

In [4]:
df.head()

,population,profit
0,6.1101,17.5920
1,5.5277,9.1302
2,8.5186,13.6620
3,7.0032,11.8540
4,5.8598,6.8233


In [5]:
df.info()
#可知两个字段均为浮点型，无null值。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   population  97 non-null     float64
 1   profit      97 non-null     float64
dtypes: float64(2)
memory usage: 1.6 KB


In [6]:
df.describe()
#profit存在负值，数据未看出异常。

,population,profit
count,97.000000,97.000000
mean,8.159800,5.839135
std,3.869884,5.510262
min,5.026900,-2.680700
25%,5.707700,1.986900
50%,6.589400,4.562300
75%,8.578100,7.046700
max,22.203000,24.147000


In [7]:
np.sum(df.duplicated(subset=['population','profit']))
#数据无重复值，无需进行重复值处理。

0

## 2）数据可视化（Visualization）

In [8]:
df.iplot(
    x='population',
    y='profit',
    xTitle='城市人口',
    yTitle='快餐车收益',
    layout=dict(height=500,width=500),
    mode='markers',size=5)

## 3）构建假设函数（Hypothesis）

单变量回归的假设 h 表示为：\\[{{h}_{\theta }}\left( x \right)={{\theta }_{0}}+{{\theta }_{1}}{x}\\] 
这个公式中有2个参数和1个变量，引入${{x}_{0}}=1$，则公式转化为：  
$${{h}_{\theta }}\left( x \right)={{\theta }^{T}}X$$其中上标T代表矩阵转置。


In [231]:
# 定义X矩阵函数
def get_X(df):
    # 引入变量x0，创建含有x0和x1的矩阵
    x_zero=pd.DataFrame(data=np.ones(len(df)))
    x=df.iloc[:, :-1]
    X=np.matrix(pd.concat([x_zero, x], axis=1))
    return X

# 定义y矩阵函数，用于之后正规方程的计算
def get_y(df):
    y=np.matrix(df.iloc[:, -1]).T
    return y

## 4）计算代价函数（Cost Function）
$$J\left( \theta  \right)=\frac{1}{2m}\sum\limits_{i=1}^{m}{{{\left( {{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}} \right)}^{2}}}$$
其中：\\[{{h}_{\theta }}\left( x \right)={{\theta }^{T}}X={{\theta }_{0}}{{x}_{0}}+{{\theta }_{1}}{{x}_{1}}+{{\theta }_{2}}{{x}_{2}}+...+{{\theta }_{n}}{{x}_{n}}\\] 

In [138]:
# 定义代价函数的计算
def computecost(X,y,theta):
    inner=np.power(((X@theta.T)-y),2)
    cost=np.sum(inner)/(2 * y.size)
    return cost

# 赋值X,y，theta初始值设为（0,0）
X=get_X(df)
y=get_y(df)
theta=np.matrix(np.zeros((1,2)))
print(X.shape, theta.shape, y.shape)

(97, 2) (1, 2) (97, 1)


In [139]:
# 计算theta为初始值（0,0）时的代价函数
computecost(X, y, theta)

32.072733877455676

## 5）梯度下降算法（Gradient Descent）
$${{\theta }_{j}}:={{\theta }_{j}}-\alpha \frac{\partial }{\partial {{\theta }_{j}}}J\left( \theta  \right)$$
简化之后得到
$${{\theta }_{j}}:={{\theta }_{j}}-\alpha \frac{1}{m}\sum\limits_{i=1}^{m}{{{\left( {{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}} \right){{x}^{(i)}}}}}$$
* 本次我们采用批量梯度下降（Batch Gradient Descent）

In [140]:
# 定义梯度下降函数,iters表示迭代次数
def gradientDescent(X, y, theta, alpha, iters):
    temp = np.matrix(np.zeros(theta.shape))#设置temp作为theta的转换
    parameters = int(theta.ravel().shape[1])#2
    cost = np.zeros(iters)
    
    # 设置迭代循环
    for i in range(iters):
        error = (X * theta.T) - y
        
        for j in range(parameters):
            term = np.multiply(error, X[:,j])
            temp[0,j] = theta[0,j] - ((alpha / len(X)) * np.sum(term))
            
        theta = temp
        cost[i] = computecost(X, y, theta)
        
    return theta, cost
# 设置alpha为0.02，iters为1000次
alpha=0.02
iters=1000


In [142]:
# 得到最终的theta
print(gradientDescent(X, y, theta, alpha, iters)[0])
# 最小的代价函数
print(gradientDescent(X, y, theta, alpha, iters)[1].min())


[[-3.78841926  1.18224801]]
4.478020743321126


## 6）代价数据可视化（visualize cost data）

In [148]:
cost=pd.DataFrame(gradientDescent(X, y, theta, alpha, iters)[1])
cost.iplot(mode='lines',
           layout=dict(height=500,width=500,xaxis=dict(title='迭代次数'),
                       yaxis=dict(title='代价函数'),title='代价数据可视化'))

* 从图中可以看出，前5次迭代，代价垂直下降。
* 迭代次数超过200次之后，代价曲线趋于平缓。

## 7）输出最终回归预测

In [199]:
final_theta_0=float(gradientDescent(X, y, theta, alpha, iters)[0][:,0])
final_theta_1=float(gradientDescent(X, y, theta, alpha, iters)[0][:,1])
df['prediction']=df['population']* final_theta_1+ final_theta_0

scatter=go.Scatter(x=df['population'].tolist(),y=df['profit'].tolist(),mode='markers',name='profit')
linear_line=go.Scatter(x=df['population'].tolist(),y=df['prediction'].tolist(),mode='lines',name='prediction')
data=[scatter,linear_line]
layout=go.Layout(height=500,width=500,xaxis=dict(title='城市人口数'),yaxis=dict(title='快餐车收益'))
figure=go.Figure(data,layout)
py.offline.iplot(figure)

* 从回归模型可看出：收益随着城市人口数增加而递增。因此可以给到老板一定的洞察。
* 可得到的业务建议：快餐车业务扩张优先选择城市人口基数大的城市。

# 2.多变量线性回归

## 1)读取数据并理解

In [200]:
df2 = pd.read_csv('D:\Learning\python\ML\code\ex1-linear regression\ex1data2.txt',names=['size of house','number of bedroom','house price'])

In [201]:
df2.head()

,size of house,number of bedroom,house price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


In [202]:
df2.info()
#可知三个字段均为整数型，无null值。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   size of house      47 non-null     int64
 1   number of bedroom  47 non-null     int64
 2   house price        47 non-null     int64
dtypes: int64(3)
memory usage: 1.2 KB


In [205]:
df2.describe()
# 数据无异常

,size of house,number of bedroom,house price
count,47.000000,47.000000,47.000000
mean,2000.680851,3.170213,340412.659574
std,794.702354,0.760982,125039.899586
min,852.000000,1.000000,169900.000000
25%,1432.000000,3.000000,249900.000000
50%,1888.000000,3.000000,299900.000000
75%,2269.000000,4.000000,384450.000000
max,4478.000000,5.000000,699900.000000


In [210]:
np.sum(df2.duplicated(subset=['size of house', 'number of bedroom', 'house price']))
#数据无重复值，无需进行重复值处理。

0

## 2）特征缩放（Feature Scaling）

In [229]:
# 设置特征缩放的函数
def normalize_feature(df2):
    df2=df2.apply(lambda column: (column - column.mean()) / column.std())
    return df2

In [230]:
df2=normalize_feature(df2)
df2.head()

,size of house,number of bedroom,house price
0,0.130010,-0.223675,0.475747
1,-0.504190,-0.223675,-0.084074
2,0.502476,-0.223675,0.228626
3,-0.735723,-1.537767,-0.867025
4,1.257476,1.090417,1.595389


## 3）梯度下降（gradient descent）

In [238]:
# 所需要的函数已在前面定义过，我们直接调用即可
X2=get_X(df2)
y2=get_y(df2)
theta2=np.matrix(np.zeros((1,3)))
alpha2=0.02
iters2=1000

# 得到最终的theta
print(gradientDescent(X2, y2, theta2, alpha2, iters2)[0])
# 最小的代价函数
print(gradientDescent(X2, y2, theta2, alpha2, iters2)[1].min())

[[-4.58262270e-17  8.84683917e-01 -5.30967489e-02]]
0.1306864834398679


In [239]:
# 代价数据可视化
cost=pd.DataFrame(gradientDescent(X2, y2, theta2, alpha2, iters2)[1])
cost.iplot(mode='lines',
           layout=dict(height=500,width=500,xaxis=dict(title='迭代次数'),
                       yaxis=dict(title='代价函数'),title='代价数据可视化'))

## 4）关于学习率（learning rate）

In [352]:
iters3=200
base=np.logspace(-5,-1,num=10,base=10)

cost=pd.DataFrame(np.zeros((iters3,10)))

for i in range(10):
    for alpha3 in base:
        cost[i] = gradientDescent(X2, y2, theta2, alpha3, iters3)[1].T
        cost.rename(columns={i:alpha3},inplace=True)
        i=i+1
    if i==10:
        break

In [353]:
data=[]
for i in cost.columns.values:
    line=go.Scatter(y=cost[i].tolist(),mode='lines',name=str(i))
    data.append(line)
    
py.offline.iplot(data)

* 从图中可以看出：不同的学习率，下降速度不同。学习率大，下降速度快。

## 5）关于标准方程（Normal Equation）
* 除了使用梯度下降解决线性回归问题外，还有一个更便捷的、不需要任何迭代的方法，那就是标准方程。
$$\theta ={{\left( {{X}^{T}}X \right)}^{-1}}{{X}^{T}}y$$
* 不过基于标准方程的计算公式，其不适用特征数过多的回归分析。

In [354]:
import numpy as np
def normalEqn(X, y):
   theta = np.linalg.inv(X.T@X)@X.T@y #X.T@X等价于X.T.dot(X)
   return theta

In [355]:
normalEqn(X2, y2)

matrix([[-5.55111512e-17],
        [ 8.84765988e-01],
        [-5.31788197e-02]])

## 鸣谢：
感谢黄海广博士提供的读书笔记及各项资料，我会在机器学习路上继续加油！